# Uppmärksamhetsmekanismer och transformatorer

En stor nackdel med rekurrenta nätverk är att alla ord i en sekvens har samma påverkan på resultatet. Detta leder till suboptimal prestanda med standard LSTM-encoder-decoder-modeller för sekvens-till-sekvens-uppgifter, såsom namngiven entity-igenkänning och maskinöversättning. I verkligheten har specifika ord i ingångssekvensen ofta större påverkan på sekventiella utgångar än andra.

Tänk på en sekvens-till-sekvens-modell, som maskinöversättning. Den implementeras med två rekurrenta nätverk, där ett nätverk (**encoder**) komprimerar ingångssekvensen till ett dolt tillstånd, och ett annat, **decoder**, vecklar ut detta dolda tillstånd till ett översatt resultat. Problemet med detta tillvägagångssätt är att nätverkets slutliga tillstånd har svårt att komma ihåg början av en mening, vilket leder till dålig modellkvalitet för långa meningar.

**Uppmärksamhetsmekanismer** ger ett sätt att vikta den kontextuella påverkan av varje ingångsvektor på varje utgångsprediktion i RNN. Detta implementeras genom att skapa genvägar mellan mellanliggande tillstånd i ingångs-RNN och utgångs-RNN. På detta sätt, när vi genererar utgångssymbolen $y_t$, tar vi hänsyn till alla dolda ingångstillstånd $h_i$, med olika viktkoefficienter $\alpha_{t,i}$.

![Bild som visar en encoder/decoder-modell med ett additivt uppmärksamhetslager](../../../../../translated_images/sv/encoder-decoder-attention.7a726296894fb567.webp)
*Encoder-decoder-modellen med additiv uppmärksamhetsmekanism i [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), citerad från [denna bloggpost](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Uppmärksamhetsmatrisen $\{\alpha_{i,j}\}$ representerar graden till vilken vissa ingångsord bidrar till genereringen av ett givet ord i utgångssekvensen. Nedan är ett exempel på en sådan matris:

![Bild som visar ett exempel på justering funnen av RNNsearch-50, hämtad från Bahdanau - arviz.org](../../../../../translated_images/sv/bahdanau-fig3.09ba2d37f202a6af.webp)

*Figur hämtad från [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Fig.3)*

Uppmärksamhetsmekanismer är ansvariga för mycket av det nuvarande eller nära nuvarande toppskiktet inom naturlig språkbehandling. Att lägga till uppmärksamhet ökar dock antalet modellparametrar avsevärt, vilket ledde till skalningsproblem med RNN:er. En viktig begränsning för att skala RNN:er är att modellernas rekurrenta natur gör det utmanande att batcha och parallellisera träning. I en RNN måste varje element i en sekvens bearbetas i sekventiell ordning, vilket innebär att det inte enkelt kan parallelliseras.

Användningen av uppmärksamhetsmekanismer i kombination med denna begränsning ledde till skapandet av de nuvarande toppmodellerna, Transformer-modeller, som vi idag känner och använder, från BERT till OpenGPT3.

## Transformer-modeller

Istället för att vidarebefordra kontexten från varje tidigare prediktion till nästa utvärderingssteg använder **transformer-modeller** **positionella kodningar** och uppmärksamhet för att fånga kontexten av en given ingång inom ett angivet textfönster. Bilden nedan visar hur positionella kodningar med uppmärksamhet kan fånga kontext inom ett givet fönster.

![Animerad GIF som visar hur utvärderingar utförs i transformer-modeller.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

Eftersom varje ingångsposition mappas oberoende till varje utgångsposition kan transformatorer parallellisera bättre än RNN:er, vilket möjliggör mycket större och mer uttrycksfulla språkmodeller. Varje uppmärksamhetshuvud kan användas för att lära sig olika relationer mellan ord, vilket förbättrar nedströmsuppgifter inom naturlig språkbehandling.

**BERT** (Bidirectional Encoder Representations from Transformers) är ett mycket stort flerskikts-transformernätverk med 12 lager för *BERT-base* och 24 för *BERT-large*. Modellen förtränas först på en stor textkorpus (Wikipedia + böcker) med hjälp av oövervakad träning (förutsäga maskerade ord i en mening). Under förträningen absorberar modellen en betydande nivå av språkförståelse som sedan kan utnyttjas med andra dataset genom finjustering. Denna process kallas **transfer learning**.

![Bild från http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/sv/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Det finns många variationer av Transformer-arkitekturer, inklusive BERT, DistilBERT, BigBird, OpenGPT3 och fler, som kan finjusteras. [HuggingFace-paketet](https://github.com/huggingface/) tillhandahåller ett bibliotek för att träna många av dessa arkitekturer med PyTorch.

## Använda BERT för textklassificering

Låt oss se hur vi kan använda en förtränad BERT-modell för att lösa vår traditionella uppgift: sekvensklassificering. Vi kommer att klassificera vårt ursprungliga AG News-dataset.

Först laddar vi HuggingFace-biblioteket och vårt dataset:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


Eftersom vi kommer att använda en förtränad BERT-modell, behöver vi använda en specifik tokenizer. Först kommer vi att ladda en tokenizer som är kopplad till den förtränade BERT-modellen.

HuggingFace-biblioteket innehåller ett arkiv med förtränade modeller, som du kan använda genom att helt enkelt ange deras namn som argument till funktionerna `from_pretrained`. Alla nödvändiga binärfiler för modellen laddas automatiskt ner.

Men ibland kan det vara nödvändigt att ladda dina egna modeller, i vilket fall du kan ange katalogen som innehåller alla relevanta filer, inklusive parametrar för tokenizer, `config.json`-filen med modellparametrar, binära vikter, etc.


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

`tokenizer`-objektet innehåller `encode`-funktionen som kan användas direkt för att koda text:


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Sedan ska vi skapa iteratorer som vi kommer att använda under träningen för att komma åt data. Eftersom BERT använder sin egen kodningsfunktion, behöver vi definiera en utfyllnadsfunktion liknande `padify` som vi har definierat tidigare:


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

I vårt fall kommer vi att använda en förtränad BERT-modell som heter `bert-base-uncased`. Låt oss ladda modellen med hjälp av paketet `BertForSequenceClassfication`. Detta säkerställer att vår modell redan har en nödvändig arkitektur för klassificering, inklusive den slutliga klassificeraren. Du kommer att se ett varningsmeddelande som anger att vikterna för den slutliga klassificeraren inte är initialiserade, och att modellen skulle kräva förträning - det är helt okej, eftersom det är precis vad vi är på väg att göra!


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

Nu är vi redo att börja träna! Eftersom BERT redan är förtränad vill vi börja med en ganska låg inlärningshastighet för att inte förstöra de initiala vikterna.

Allt hårt arbete utförs av modellen `BertForSequenceClassification`. När vi kör modellen på träningsdata returnerar den både förlusten och nätverksutgången för den aktuella minibatchen. Vi använder förlusten för parameteroptimering (`loss.backward()` utför bakåtpasseringen) och `out` för att beräkna träningsnoggrannheten genom att jämföra de erhållna etiketterna `labs` (beräknade med `argmax`) med de förväntade `labels`.

För att kontrollera processen ackumulerar vi förlust och noggrannhet över flera iterationer och skriver ut dem var `report_freq` träningscykel.

Den här träningen kommer sannolikt att ta ganska lång tid, så vi begränsar antalet iterationer.


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


Du kan se (särskilt om du ökar antalet iterationer och väntar tillräckligt länge) att BERT-klassificering ger oss ganska bra noggrannhet! Det beror på att BERT redan förstår språkets struktur ganska väl, och vi behöver bara finjustera den slutliga klassificeraren. Men eftersom BERT är en stor modell tar hela träningsprocessen lång tid och kräver betydande beräkningskraft! (GPU, och helst fler än en).

> **Note:** I vårt exempel har vi använt en av de minsta förtränade BERT-modellerna. Det finns större modeller som sannolikt ger bättre resultat.


## Utvärdera modellens prestanda

Nu kan vi utvärdera modellens prestanda på testdatamängden. Utvärderingsloopen liknar träningsloopen, men vi får inte glömma att sätta modellen i utvärderingsläge genom att kalla på `model.eval()`.


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## Viktiga punkter

I den här enheten har vi sett hur enkelt det är att ta en förtränad språkmodell från **transformers**-biblioteket och anpassa den till vår textklassificeringsuppgift. På samma sätt kan BERT-modeller användas för entity extraction, frågehantering och andra NLP-uppgifter.

Transformer-modeller representerar det nuvarande toppskiktet inom NLP, och i de flesta fall bör de vara den första lösningen du börjar experimentera med när du implementerar skräddarsydda NLP-lösningar. Men att förstå de grundläggande principerna bakom rekurrenta neurala nätverk som diskuterats i denna modul är oerhört viktigt om du vill bygga avancerade neurala modeller.



---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, bör du vara medveten om att automatiska översättningar kan innehålla fel eller inexaktheter. Det ursprungliga dokumentet på dess originalspråk bör betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för eventuella missförstånd eller feltolkningar som uppstår vid användning av denna översättning.
